    This IPYNB file contain interactive plots and graphs for understanding of the data.

# Imports

In [1]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Functions

In [2]:
def RSI(series, period):
    # Calculate price changes
    delta = series.diff(1)
    
    # Separate positive and negative price changes
    gain = delta.where(delta > 0, 0)
    loss = -delta.where(delta < 0, 0)
    
    # Calculate average gain and average loss over the specified period
    avg_gain = gain.rolling(window=period, min_periods=1).mean()
    avg_loss = loss.rolling(window=period, min_periods=1).mean()
    
    # Calculate relative strength (RS)
    rs = avg_gain / avg_loss
    
    # Calculate RSI
    rsi = 100 - (100 / (1 + rs))
    
    return rsi

# Minute-by-minute Data Visualization

In [3]:
# Load data
df = pd.read_csv('data/stock/CC_1min.txt', sep=',', index_col=False, header=None, names=['date', 'open', 'high', 'low', 'close', 'volume'])
# Convert the 'date' column to a datetime object
df['date'] = pd.to_datetime(df['date'])

# Drop after hours data
df = df[df['date'].dt.time >= pd.to_datetime('09:30:00').time()] 
df = df[df['date'].dt.time <= pd.to_datetime('16:00:00').time()]

# Calculate RSI
df['RSI'] = RSI(df['close'], 10)

### Check the data

In [4]:
df.head(1)

,date,open,high,low,close,volume,RSI
0,2015-06-19 09:49:00,16.5264,16.5264,16.5264,16.5264,100,NaN


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 639014 entries, 0 to 647029
Data columns (total 7 columns):
 #   Column  Non-Null Count   Dtype         
---  ------  --------------   -----         
 0   date    639014 non-null  datetime64[ns]
 1   open    639014 non-null  float64       
 2   high    639014 non-null  float64       
 3   low     639014 non-null  float64       
 4   close   639014 non-null  float64       
 5   volume  639014 non-null  int64         
 6   RSI     638883 non-null  float64       
dtypes: datetime64[ns](1), float64(5), int64(1)
memory usage: 39.0 MB


### Plots

In [6]:
# Plot for most recent day
df_plot = df[-300:]

START_DATE = df_plot['date'].iloc[0]
END_DATE = df_plot['date'].iloc[-1]

# Create subplots
fig = make_subplots(rows=2, cols=1, shared_xaxes=True, vertical_spacing=0.05,
                    subplot_titles=("Price", "Volume and RSI"),
                    row_width=[0.25, 0.7],  
                    specs=[[{"secondary_y": False}], [{"secondary_y": True}]]) # secondary_y: whether or not to display a secondary y-axis



# Subplot 1: Candlestick chart with Bollinger Bands
fig.add_trace(go.Candlestick(
    x=df_plot['date'],
    open=df_plot['open'],
    high=df_plot['high'],
    low=df_plot['low'],
    close=df_plot['close'],
    name='Price'
), row=1, col=1)

fig.update_xaxes(type='category', row=1, col=1) # Need to correct x axis since time is not continuous for minute data, time is missing for weekends
fig.update_yaxes(tickprefix="$", row=1, col=1) # Add dollar sign to yaxis tick values for the first subplot


# Subplot 2: Volume and RSI with two y-axes
fig.add_trace(go.Bar(
    x=df_plot['date'],
    y=df_plot['volume'],  # Assuming volume is in millions
    marker_color='blue',
    name='Volume',
    opacity=0.8,
    yaxis='y1'
), row=2, col=1)

fig.add_trace(go.Scatter(
    x=df_plot['date'],
    y=df_plot['RSI'],
    line=dict(color='orange', width=1),
    name='RSI',
    opacity=1,
    yaxis='y2',
), row=2, col=1, secondary_y=True)

# Update y-axes for the second subplot
fig.update_yaxes(title_text="Volume", row=2, col=1, secondary_y=False)
fig.update_yaxes(title_text="RSI", row=2, col=1, secondary_y=True)

fig.update_layout(
    hovermode = "x unified", # Display x-axis hover information of all subplots at once
)
fig.update_traces(xaxis='x1') 

# fig.update_xaxes(showticklabels=False, row=2, col=1, type='category') # Need to correct x axis since time is not continuous for minute data, time is missing for weekends

# Add custom x ticks
fig.update_xaxes(
    row=2, col=1, showgrid=True,
    ticktext=["09:30", "10:30", "11:30", "12:30", "13:30", "14:30", "15:30"],
    tickvals=["09:30", "10:30", "11:30", "12:30", "13:30", "14:30", "15:30"]
)

# Set layout options
fig.update_layout(
    title='Price, Volume, and RSI from <b style="color:red">{}</b> to <b style="color:red">{}</b>'.format(START_DATE, END_DATE),
    xaxis=dict(rangeslider=dict(visible=False)),
    showlegend=True,
    height=1000,
)

# Show the figure
fig.show()

# Day-by-day Data Visualization

### Converting to daily data

In [7]:
# Read the data
df = pd.read_csv('data/stock/CC_1min.txt', sep=',', index_col=False, header=None, names=['date', 'open', 'high', 'low', 'close', 'volume'])

# Convert the 'date' column to a datetime object
df['date'] = pd.to_datetime(df['date'])

# Resample the data to daily OHLCV using the 'date' column as the index
df_resampled = df.resample('D', on='date').agg({'open':'first', 'high':'max', 'low':'min', 'close':'last', 'volume':'sum'})

# Reset the index to have the date as a separate column
df_resampled.reset_index(inplace=True)

# Remove rows with NaN values
df_resampled.dropna(subset=['open', 'high', 'low', 'close', 'volume'], inplace=True)

### Adding Features

    Date, Day, Month, Year, Day of Week

In [8]:
# Add additional date-related columns if needed
df_resampled['year'] = df_resampled['date'].dt.year
df_resampled['month'] = df_resampled['date'].dt.month
df_resampled['day'] = df_resampled['date'].dt.day
df_resampled['day_of_week'] = df_resampled['date'].dt.dayofweek

    RSI, Bollinger Bands, EMA

In [9]:
df_resampled['RSI'] = RSI(df_resampled['close'], 14)
df_resampled['RSI'].fillna(0, inplace=True)

# EMA
df_resampled['EMA_5'] = df_resampled['close'].ewm(span=10, adjust=False).mean()
df_resampled['EMA_10'] = df_resampled['close'].ewm(span=20, adjust=False).mean()
df_resampled['EMA_20'] = df_resampled['close'].ewm(span=50, adjust=False).mean()

# Bolliger Bands
time_period = 20
stdev_factor = 2
df_resampled['MA'] = df_resampled['close'].rolling(window=time_period).mean()
df_resampled['BB_up'] = df_resampled['MA'] + stdev_factor * df_resampled['close'].rolling(window=time_period).std()
df_resampled['BB_down'] = df_resampled['MA'] - stdev_factor * df_resampled['close'].rolling(window=time_period).std()

# Reset the index to be continous
df_resampled.reset_index(drop=True, inplace=True)

### Check Data

In [10]:
df_resampled.head(1)

,date,open,high,low,close,volume,year,month,day,day_of_week,RSI,EMA_5,EMA_10,EMA_20,MA,BB_up,BB_down
0,2015-06-19,16.5264,17.5101,16.1329,16.4084,2633685,2015,6,19,4,0.0,16.4084,16.4084,16.4084,NaN,NaN,NaN


In [11]:
df_resampled.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1685 entries, 0 to 1684
Data columns (total 17 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   date         1685 non-null   datetime64[ns]
 1   open         1685 non-null   float64       
 2   high         1685 non-null   float64       
 3   low          1685 non-null   float64       
 4   close        1685 non-null   float64       
 5   volume       1685 non-null   int64         
 6   year         1685 non-null   int32         
 7   month        1685 non-null   int32         
 8   day          1685 non-null   int32         
 9   day_of_week  1685 non-null   int32         
 10  RSI          1685 non-null   float64       
 11  EMA_5        1685 non-null   float64       
 12  EMA_10       1685 non-null   float64       
 13  EMA_20       1685 non-null   float64       
 14  MA           1666 non-null   float64       
 15  BB_up        1666 non-null   float64       
 16  BB_dow

## Plots

In [12]:
df_plot = df_resampled.iloc[-1000:] # Plot only the last 1000 days

START_DATE = df_plot['date'].iloc[0]
END_DATE = df_plot['date'].iloc[-1]

# Create subplots
fig = make_subplots(rows=2, cols=1, shared_xaxes=True, vertical_spacing=0.05,
                    subplot_titles=("Price and Bollinger Bands", "Volume and RSI"),
                    row_width=[0.25, 0.7],
                    specs=[[{"secondary_y": False}], [{"secondary_y": True}]]) # secondary_y: whether or not to display a secondary y-axis



# Subplot 1: Candlestick chart with Bollinger Bands
fig.add_trace(go.Candlestick(
    x=df_plot['date'],
    open=df_plot['open'],
    high=df_plot['high'],
    low=df_plot['low'],
    close=df_plot['close'],
    name='Price'
), row=1, col=1)

fig.add_trace(go.Scatter(
    x=df_plot['date'],
    y=df_plot['BB_up'],
    line=dict(color='gray', width=1),
    name='BB_up',
    opacity=0.0
), row=1, col=1)

fig.add_trace(go.Scatter(
    x=df_plot['date'],
    y=df_plot['BB_down'],
    line=dict(color='gray', width=1),
    name='BB_down',
    opacity=0.0,
    fill='tonexty',  # Fill between BB_up and BB_down
    fillcolor='rgba(0,100,80,0.2)'
), row=1, col=1)

fig.update_yaxes(tickprefix="$", row=1, col=1) # Add dollar sign to yaxis tick values for the first subplot



# Subplot 2: Volume and RSI with two y-axes
fig.add_trace(go.Bar(
    x=df_plot['date'],
    y=df_plot['volume'], 
    marker_color='blue',
    name='Volume',
    opacity=1
), row=2, col=1)

fig.add_trace(go.Scatter(
    x=df_plot['date'],
    y=df_plot['RSI'],
    line=dict(color='orange', width=1),
    name='RSI',
    opacity=1
), row=2, col=1, secondary_y=True)

# Update y-axes for the second subplot
fig.update_yaxes(title_text="Volume", row=2, col=1)
fig.update_yaxes(title_text="RSI", row=2, col=1, secondary_y=True)

fig.update_layout(
    hovermode = "x unified",
    legend_traceorder="normal"
)
fig.update_traces(xaxis='x1')


# Set layout options
fig.update_layout(
    title='Price, Bollinger Bands, Volume, and RSI from <b style="color:red">{}</b> to <b style="color:red">{}</b>'.format(START_DATE.strftime('%Y-%m-%d'),END_DATE.strftime('%Y-%m-%d')),
    xaxis=dict(rangeslider=dict(visible=False)),
    height=800
)

# Show the figure
fig.show()

# EDA

    This explores importance of day of week on price change

In [13]:
price_change_data = []

# Calculate the average close - open price change for each day of the week
for i in range(7):
    avg_price_change = df_resampled[df_resampled['day_of_week'] == i]['close'].mean() - df_resampled[df_resampled['day_of_week'] == i]['open'].mean()

    avg_price_rise = df_resampled[(df_resampled['day_of_week'] == i) & (df_resampled['close'] > df_resampled['open'])]['close'].mean() - df_resampled[(df_resampled['day_of_week'] == i) & (df_resampled['close'] > df_resampled['open'])]['open'].mean()
    avg_price_fall = df_resampled[(df_resampled['day_of_week'] == i) & (df_resampled['close'] < df_resampled['open'])]['close'].mean() - df_resampled[(df_resampled['day_of_week'] == i) & (df_resampled['close'] < df_resampled['open'])]['open'].mean()
    
    num_rise = len(df_resampled[(df_resampled['day_of_week'] == i) & (df_resampled['close'] > df_resampled['open'])])
    num_fall = len(df_resampled[(df_resampled['day_of_week'] == i) & (df_resampled['close'] < df_resampled['open'])])
    
    avg_volume = df_resampled[df_resampled['day_of_week'] == i]['volume'].mean()
    
    price_change_data.append({'day_of_week': i, 'price_change': avg_price_change, 'price_rise': avg_price_rise, 'price_fall': avg_price_fall, 'num_rise': num_rise, 'num_fall': num_fall, 'volume': avg_volume})


# Create a DataFrame from the list of dictionaries
df_price_change = pd.DataFrame(price_change_data)

df_price_change

,day_of_week,price_change,price_rise,price_fall,num_rise,num_fall,volume
0,0,-0.016571,0.524829,-0.537876,154,160,1.997417e+06
1,1,-0.026054,0.523718,-0.545305,165,175,2.120766e+06
2,2,-0.013183,0.454745,-0.506488,176,167,2.110446e+06
3,3,-0.048234,0.489925,-0.593047,171,169,2.086256e+06
4,4,0.004023,0.522102,-0.483864,162,172,2.275375e+06
5,5,NaN,NaN,NaN,0,0,NaN
6,6,NaN,NaN,NaN,0,0,NaN


# Resources Used

- https://pytorch.org/docs/stable/index.html

- https://matplotlib.org/stable/index.html

- https://pandas.pydata.org/docs/

- https://numpy.org/doc/stable/

- https://github.com/xljiang/stock-market-simulator